In [45]:
using JuMP
using MosekTools, SCS
using LinearAlgebra
using ControlSystemsBase

In [53]:
@variable(model, P[i=1:n, j=1:n], PSD)
@constraint(model, P >= 0, PSDCone())
@constraint(model, A'*P + P*A >= -Q, PSDCone())
@constraint(model, A'*P + P*A <= -Q, PSDCone());

In [39]:
set_silent(model)
optimize!(model);
P = value.(P);
# Test result:
A'*P + P*A ≈ -Q

true

In [55]:
A = [1 0.1; 0 1];
B = [0; 1];
sys = ss(A,B,I,0,1)
Q = I
R = I

2.0

In [122]:
K₁ = lqr(sys,Q,R)

1×2 Matrix{Float64}:
 0.589088  0.711884

In [128]:
sys_c= d2c(sys)
Ac = sys_c.A
Bc = sys_c.B
K₂ = lqr(sys_c,Q,R)

1×2 Matrix{Float64}:
 1.0  1.14659

In [129]:
eigen(Ac-Bc*K₂).values

2-element Vector{Float64}:
 -0.9962046266657549
 -0.10038098330731127

In [141]:
model = Model(Mosek.Optimizer);
n = 2
@variable(model, Σ[i=1:n, j=1:n], PSD)
@variable(model, Z[i=1:1, j=1:1], PSD)
@variable(model, L[i=1:1, j=1:n])

@objective(model, Min, tr(Q*Σ)+tr(R*Z))
@constraint(model, Σ-1e-5*I >= 0, PSDCone())
# Z >= KΓK'
@constraint(model, [Z L; L' Σ] >= 0, PSDCone())
# Lyupanov
@constraint(model, Ac*Σ + Bc*L + (Ac*Σ + Bc*L)'+3I >= 0, PSDCone())
@constraint(model, Ac*Σ + Bc*L + (Ac*Σ + Bc*L)'+3I <= 0, PSDCone());

In [142]:
set_silent(model)
optimize!(model);

In [143]:
Σ, Z, L = value.(Σ), value.(Z), value.(L)

([6.563020716823327 -5.2638488352766535; -5.2638488352766535 5.026978630648215], [1.1011068507989084;;], [-0.5276978606399902 -0.4999999937871439])

In [144]:
K = L * inv(Σ)
eigen(Ac+Bc*K).values

2-element Vector{Float64}:
 -0.99631318577213
 -0.10038125113819096

In [115]:
eigen(Σ-I-(A+B*K)*inv(Γ)*(A+B*K)')

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
2-element Vector{Float64}:
 1.2523844287250598e-8
 1.5829566948841742e-8
vectors:
2×2 Matrix{Float64}:
 -0.935365  -0.353684
 -0.353684   0.935365

In [82]:
tr(Q*Σ)+tr(R*Z)

2.0000003221936478

In [102]:
model = Model(Mosek.Optimizer);
n = 2
@variable(model, X[i=1:n, j=1:n], PSD)
@variable(model, Z[i=1:n, j=1:n], PSD)
@variable(model, L[i=1:1, j=1:n])
@objective(model, Min, tr(Z))
@constraint(model, X-1e-5*I >= 0, PSDCone())
# Z >= X^{-1}   
@constraint(model, [Z I; I X] >= 0, PSDCone())
# Lyupanov
@constraint(model, [X (A*X+B*L)'; (A*X+B*L) X] >= 0, PSDCone());

In [103]:
set_silent(model)
optimize!(model);

In [104]:
L, X, Z = value.(L), value.(X), value.(Z)

([-1.4902018149978286e6 -3.842934205993686e7], [5.404510815271053e7 -1.7836972517953224e7; -1.7836972517953224e7 6.041490172454586e7], [2.327537352178023e-8 5.553688204393768e-9; 5.553688204393768e-9 2.120936661759748e-8])

In [105]:
K = L * inv(X) 

1×2 Matrix{Float64}:
 -0.263149  -0.713783

In [107]:
eigen(A+B*K).values

2-element Vector{Float64}:
 0.325214498168043
 0.961002535440085